In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import pickle
import re
import os
import math

In [2]:
ACTION_FILE = './datafile/JData_Action.csv'
COMMENT_FILE = './datafile/JData_Comment.csv'
PRODUCT_FILE = './datafile/JData_Product.csv'
USER_FILE = './datafile/JData_User.csv'

In [12]:
#统计所有购买过第八类商品的用户关于年龄的关系
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-time-type-user_id','rb') as f:
#     dataset = pickle.load(f)
# user_info = pd.read_csv(USER_FILE,encoding='gbk')
# dataset = pd.DataFrame(dataset[(dataset.type==4)&(dataset.cate==8)]['user_id']).drop_duplicates()         #12921
# dataset = pd.merge(dataset,user_info,on='user_id',how='left')
# len(dataset[dataset.age=='-1'])              #1435
# len(dataset[dataset.age=='16-25岁'])            #1083
# len(dataset[dataset.age=='26-35岁'])           #5938
# len(dataset[dataset.age=='36-45岁'])          #3875
# len(dataset[dataset.age=='46-55岁'])           #330
# 1435/12921             #0.111
# 1083/12921             #0.0838
# 5938/12921             #0.4595
# 3875/12921             #0.2998
# 330/12921              #0.0255

#所有用户中
# dataset = dataset[dataset['cate']==8][['user_id']].drop_duplicates()
# dataset = pd.merge(dataset,user_info,on='user_id',how='left')
# len(dataset[dataset.age=='-1'])              #14315
# len(dataset[dataset.age=='16-25岁'])            #8759
# len(dataset[dataset.age=='26-35岁'])           #46340
# len(dataset[dataset.age=='36-45岁'])          #30149
# len(dataset[dataset.age=='46-55岁'])           #3299

1435/14315                  #0.1002
1083/8759                   #0.1236
5938/46340                  #0.1281
3875/30149                  #0.1285


0.12852830939666324

# 第一步，找出潜在购买用户

In [56]:

# test_set = pd.read_csv('have_cate_8_browse_record.csv')        #1107
# test_set[test_set['8/count']>0.63]        #607    0.63为平均值  
# test_set[test_set['8/count']>0.1]           #1004
# test_set[test_set['8/count']>0.2]             #903
# test_set[test_set['8/count']>0.3]            #826
# test_set[test_set['8/count']>0.4]              #750
# test_set[test_set['8/count']>0.5]                #674
# test_set[test_set['8/count']>0.6]               #616
# test_set[test_set['8/count']>0.7]                 #572
# test_set[test_set['8/count']>0.8]                   #519
# test_set[test_set['8/count']>0.9]                   #429
# test_set[test_set['8/count']==1]                    #324
# user_load_browse_frame = pd.read_csv('load_frame.csv')
# user_load_browse_frame


#登录天数
# test_user_id = pd.DataFrame(test_set['user_id'])
# test_user_frame = pd.merge(test_user_id,user_load_browse_frame,on='user_id',how='left')
# test_user_frame = test_user_frame.dropna(axis=0)   #平均登录天数为5.8天，方差6.5     总共1080条
# len(test_user_frame[test_user_frame['load_days']<=6])         #770
# len(test_user_frame[test_user_frame['load_days']<=10])        #920
# len(test_user_frame[test_user_frame['load_days']<=15])        #1007
# len(test_user_frame[test_user_frame['load_days']<=7])         #824
# len(test_user_frame[test_user_frame['load_days']<=5])         #713
# len(test_user_frame[test_user_frame['load_days']<=4])         #650
# len(test_user_frame[test_user_frame['load_days']<=3])         #543
# len(test_user_frame[test_user_frame['load_days']<=2])         #399
# len(test_user_frame[test_user_frame['load_days']<=1])         #197


#全部登录信息
# with open('./cache/Action_2016-02-01_2016-04-11-buy_user_load_days','rb') as f:
#     user_load_info = pickle.load(f)
#登录过于频繁的购买几率低得可怜
# len(user_load_info[user_load_info['load_days']<=15])         #57822,总共7w8    
# len(user_load_info[user_load_info['load_days']<=10])         #42137
# len(user_load_info[user_load_info['load_days']<=7])          #28689
# len(user_load_info[user_load_info['load_days']<=6])          #23707
# len(user_load_info[user_load_info['load_days']<=5])          #18610
# len(user_load_info[user_load_info['load_days']<=4])          #13475
# len(user_load_info[user_load_info['load_days']<=3])          #8853
# len(user_load_info[user_load_info['load_days']<=2])          #4907
# len(user_load_info[user_load_info['load_days']<=1])          #1810



# #以登录天数7为极限，求取准确率,通过正推和反推两张方法校正，进而构建模型
# part_test_set = test_set[['user_id','cate_8','browse_count','8/count']]
# test_user_frame = pd.merge(test_user_frame,part_test_set,on='user_id',how='left')
# test_load_less_seven_days_user = test_user_frame[test_user_frame['load_days']<=7]                       #824
# len(test_load_less_seven_days_user[test_load_less_seven_days_user['last_buy_time']=='2016-01-30'])      #667
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==1)&\
#                                    (test_load_less_seven_days_user['last_buy_time']=='2016-01-30')])    #161
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==2)&\
#                                    (test_load_less_seven_days_user['last_buy_time']=='2016-01-30')])    #172
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==3)&\
#                                    (test_load_less_seven_days_user['last_buy_time']=='2016-01-30')])    #121
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==4)&\
#                                    (test_load_less_seven_days_user['last_buy_time']=='2016-01-30')])    #86
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==5)&\
#                                    (test_load_less_seven_days_user['last_buy_time']=='2016-01-30')])    #51
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==6)&\
#                                    (test_load_less_seven_days_user['last_buy_time']=='2016-01-30')])    #40
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==7)&\
#                                    (test_load_less_seven_days_user['last_buy_time']=='2016-01-30')])    #36
#总结：1-31号之后从未买过商品的用户，登录次数越多，购买人数越少


# len(test_load_less_seven_days_user[test_load_less_seven_days_user['last_buy_time']!='2016-01-30'])      #157  
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==1)&\
#                                    (test_load_less_seven_days_user['last_buy_time']!='2016-01-30')])    #36
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==2)&\
#                                    (test_load_less_seven_days_user['last_buy_time']!='2016-01-30')])    #30
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==3)&\
#                                    (test_load_less_seven_days_user['last_buy_time']!='2016-01-30')])    #23
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==4)&\
#                                    (test_load_less_seven_days_user['last_buy_time']!='2016-01-30')])    #21
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==5)&\
#                                    (test_load_less_seven_days_user['last_buy_time']!='2016-01-30')])    #12
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==6)&\
#                                    (test_load_less_seven_days_user['last_buy_time']!='2016-01-30')])    #17
# len(test_load_less_seven_days_user[(test_load_less_seven_days_user['load_days']==7)&\
#                                    (test_load_less_seven_days_user['last_buy_time']!='2016-01-30')])    #18
#总结:有过购买记录的用户中寻找在接下来几天依然会购买的概率偏低

#查看登陆天数小于7的用户的完整数据
# with open('./cache/Action_2016-02-01_2016-04-11-user_id-cate-time-type','rb') as f:
#     user_dataset = pickle.load(f)
# test_load_less_seven_days_user = pd.merge(test_load_less_seven_days_user,user_dataset,on='user_id',how='left')
# test_load_less_seven_days_user = test_load_less_seven_days_user[test_load_less_seven_days_user['cate']==8]
# len(pd.DataFrame(test_load_less_seven_days_user[test_load_less_seven_days_user['time']>'2016-03-27 00:00:00']\
#                  ['user_id']).drop_duplicates())                                        #664
# len(pd.DataFrame(test_load_less_seven_days_user[test_load_less_seven_days_user['time']>'2016-04-01 00:00:00']\
#                  ['user_id']).drop_duplicates())                                          #621
# len(pd.DataFrame(test_load_less_seven_days_user[test_load_less_seven_days_user['time']>'2016-04-04 00:00:00']\
#                  ['user_id']).drop_duplicates())                                        #588
# len(pd.DataFrame(test_load_less_seven_days_user[test_load_less_seven_days_user['time']>'2016-04-08 00:00:00']\
#                  ['user_id']).drop_duplicates())                                        #444
#总结：选择最后7天登陆了再购买的几率大些

# len(pd.DataFrame(test_load_less_seven_days_user[test_load_less_seven_days_user['type']==2]['user_id'].drop_duplicates()))   #412
# len(pd.DataFrame(test_load_less_seven_days_user[test_load_less_seven_days_user['type']==3]['user_id'].drop_duplicates()))   #133
# len(pd.DataFrame(test_load_less_seven_days_user[test_load_less_seven_days_user['type']==5]['user_id'].drop_duplicates()))   #108
# len(pd.DataFrame(test_load_less_seven_days_user[(test_load_less_seven_days_user['type']==2)|\
#         (test_load_less_seven_days_user['type']==5)|(test_load_less_seven_days_user['type']==3)]['user_id'].drop_duplicates()))#469

# temp = pd.DataFrame(test_load_less_seven_days_user[(test_load_less_seven_days_user['type']==2)|\
#         (test_load_less_seven_days_user['type']==5)|(test_load_less_seven_days_user['type']==3)]['user_id'].drop_duplicates())
# temp['label'] = 0
# just_browse_user = pd.merge(test_load_less_seven_days_user,temp,on='user_id',how='left')
# just_browse_user = just_browse_user.fillna('retain')
# just_browse_user_id = just_browse_user[just_browse_user['label']=='retain']



#剩余40%是只有浏览记录，没有加购，删除，收藏行为的用户                                         #355
# temp_1 = just_browse_user_id[['user_id','load_days','time']].drop_duplicates()              #平均登录2.7天
# temp_1 = temp_1.groupby(['user_id','load_days'],as_index=False).time.max()
# len(pd.DataFrame(temp_1[temp_1['time']>'2016-03-27 00:00:00']['user_id']).drop_duplicates()) #250条用户是在最后15天有浏览记录存在的
# len(pd.DataFrame(temp_1[temp_1['time']>'2016-04-04 00:00:00']['user_id']).drop_duplicates()) #215个用户是在最后七天有浏览记录存在的
# len(pd.DataFrame(temp_1[temp_1['time']>'2016-04-06 00:00:00']['user_id']).drop_duplicates()) #188          5天
# len(pd.DataFrame(temp_1[temp_1['time']>'2016-04-07 00:00:00']['user_id']).drop_duplicates()) #177          4天
# len(pd.DataFrame(temp_1[temp_1['time']>'2016-04-08 00:00:00']['user_id']).drop_duplicates()) #146      最后3天
# len(pd.DataFrame(temp_1[temp_1['time']>'2016-04-09 00:00:00']['user_id']).drop_duplicates()) #107          2天
# len(pd.DataFrame(temp_1[temp_1['time']>'2016-04-10 00:00:00']['user_id']).drop_duplicates()) #58           1天

#登陆时间和登录天数的间接关系
# # temp_2 = temp_1[temp_1['time']>'2016-04-04 00:00:00']
# len(pd.DataFrame(temp_2[temp_2['load_days']<=7]['user_id']).drop_duplicates())                #215
# len(pd.DataFrame(temp_2[temp_2['load_days']<=6]['user_id']).drop_duplicates())                #204
# len(pd.DataFrame(temp_2[temp_2['load_days']<=5]['user_id']).drop_duplicates())                #189
# len(pd.DataFrame(temp_2[temp_2['load_days']<=4]['user_id']).drop_duplicates())                #173
# len(pd.DataFrame(temp_2[temp_2['load_days']<=3]['user_id']).drop_duplicates())                #157
# len(pd.DataFrame(temp_2[temp_2['load_days']<=2]['user_id']).drop_duplicates())                #123
# len(pd.DataFrame(temp_2[temp_2['load_days']<=1]['user_id']).drop_duplicates())                #58 


# temp_3 = temp_1[temp_1['time']>'2016-04-06 00:00:00']
# len(pd.DataFrame(temp_3[temp_3['load_days']<=7]['user_id']).drop_duplicates())                 #188
# len(pd.DataFrame(temp_3[temp_3['load_days']<=6]['user_id']).drop_duplicates())                 #178
# len(pd.DataFrame(temp_3[temp_3['load_days']<=5]['user_id']).drop_duplicates())                 #166
# len(pd.DataFrame(temp_3[temp_3['load_days']<=4]['user_id']).drop_duplicates())                 #153
# len(pd.DataFrame(temp_3[temp_3['load_days']<=3]['user_id']).drop_duplicates())                 #140
# len(pd.DataFrame(temp_3[temp_3['load_days']<=2]['user_id']).drop_duplicates())                 #109
# len(pd.DataFrame(temp_3[temp_3['load_days']<=1]['user_id']).drop_duplicates())                 #50


# temp_4 = temp_1[temp_1['time']>'2016-04-10 00:00:00']
# len(pd.DataFrame(temp_4[temp_4['load_days']<=7]['user_id']).drop_duplicates())                  #58
# len(pd.DataFrame(temp_4[temp_4['load_days']<=6]['user_id']).drop_duplicates())                  #56
# len(pd.DataFrame(temp_4[temp_4['load_days']<=5]['user_id']).drop_duplicates())                  #52
# len(pd.DataFrame(temp_4[temp_4['load_days']<=4]['user_id']).drop_duplicates())                  #45
# len(pd.DataFrame(temp_4[temp_4['load_days']<=3]['user_id']).drop_duplicates())                  #40
# len(pd.DataFrame(temp_4[temp_4['load_days']<=2]['user_id']).drop_duplicates())                  #28
# len(pd.DataFrame(temp_4[temp_4['load_days']<=1]['user_id']).drop_duplicates())                  #15

# test_load_less_seven_days_user = test_load_less_seven_days_user[test_load_less_seven_days_user['time']>'2016-04-04 00:00:00']
# len(pd.DataFrame(test_load_less_seven_days_user[(test_load_less_seven_days_user['type']==2)|\
#         (test_load_less_seven_days_user['type']==5)|(test_load_less_seven_days_user['type']==3)]['user_id'].drop_duplicates()))#325

#最后三天
# test_load_less_seven_days_user = test_load_less_seven_days_user[test_load_less_seven_days_user['time']>'2016-04-08 00:00:00']
# len(pd.DataFrame(test_load_less_seven_days_user[(test_load_less_seven_days_user['type']==2)|\
#         (test_load_less_seven_days_user['type']==5)|(test_load_less_seven_days_user['type']==3)]['user_id'].drop_duplicates()))#254

#登录天数小于7天的用户登录信息
# all_load_less_seven_days_user_info = user_load_info[user_load_info['load_days']<=7]
# all_load_less_seven_days_user_info = all_load_less_seven_days_user_info[['user_id','load_days']]
# all_load_less_seven_days_user_info = pd.merge(all_load_less_seven_days_user_info,user_dataset,on='user_id',how='left')
# all_load_less_seven_days_user_info = all_load_less_seven_days_user_info[all_load_less_seven_days_user_info['cate']==8]
# len(pd.DataFrame(all_load_less_seven_days_user_info[all_load_less_seven_days_user_info['time']>'2016-04-04 00:00:00']\
#                  ['user_id']).drop_duplicates())                                            #10845

#登陆时间大于16-04-04 00:00:00
# all_load_less_seven_days_user_info = all_load_less_seven_days_user_info[all_load_less_seven_days_user_info['time']>'2016-04-04 00:00:00']
# with open('./cache/Action_2016-02-01_2016-04-11-load_less_eight_days','wb') as f:
#     pickle.dump(all_load_less_seven_days_user_info,f)
# len(pd.DataFrame(all_load_less_seven_days_user_info[(all_load_less_seven_days_user_info['type']==2)|(all_load_less_seven_days_user_info['type']==3)|
#                                    (all_load_less_seven_days_user_info['type']==5)]['user_id']).drop_duplicates())  #3420条，有325条是会购买的
#最后三天加入购物车
# all_load_less_seven_days_user_info_1 = all_load_less_seven_days_user_info[all_load_less_seven_days_user_info['time']>'2016-04-08 00:00:00']
# len(pd.DataFrame(all_load_less_seven_days_user_info_1[(all_load_less_seven_days_user_info_1['type']==2)|(all_load_less_seven_days_user_info_1['type']==3)|
#                                    (all_load_less_seven_days_user_info_1['type']==5)]['user_id']).drop_duplicates())      #1993     #254











#选择登陆次数少于7天，并且在最后7天登陆过得用户
# test_load_less_seven_days_user = test_load_less_seven_days_user[test_load_less_seven_days_user['time']>'2016-04-04 00:00:00']
# test_load_less_seven_days_user = test_load_less_seven_days_user[['user_id','cate_8','browse_count','load_days','8/count','time','type']]
# USER_INFO = pd.read_csv(USER_FILE,encoding='gbk')
# test_load_less_seven_days_user = pd.merge(test_load_less_seven_days_user,USER_INFO,on='user_id',how='left')
#年龄关系
# test_load_less_seven_days_user['age_0'] = 0
# test_load_less_seven_days_user['age_1'] = 0
# test_load_less_seven_days_user['age_2'] = 0
# test_load_less_seven_days_user['age_3'] = 0
# test_load_less_seven_days_user['age_4'] = 0
# test_load_less_seven_days_user_1 = test_load_less_seven_days_user
# a =test_load_less_seven_days_user_1[test_load_less_seven_days_user_1['age']=='-1']
# a['age_0'] = 2
# b =test_load_less_seven_days_user_1[test_load_less_seven_days_user_1['age']=='16-25岁']
# b['age_1']=1
# c =test_load_less_seven_days_user_1[test_load_less_seven_days_user_1['age']=='26-35岁']
# c['age_2'] = 3
# d =test_load_less_seven_days_user_1[test_load_less_seven_days_user_1['age']=='36-45岁']
# d['age_3'] = 4
# e =test_load_less_seven_days_user_1[test_load_less_seven_days_user_1['age']=='46-55岁']
# e['age_4'] = 0
# test_load_less_seven_days_user_1 = pd.concat([a,b,c,d,e],axis=0)


#购买用户与性别之间的联系
# user_sex = test_load_less_seven_days_user_1[['user_id','sex']].drop_duplicates()
# len(user_sex[user_sex['sex']==0])   #214      男        2
# len(user_sex[user_sex['sex']==1])   #49    女           1
# len(user_sex[user_sex['sex']==2])   #318   未知         3
# aa = test_load_less_seven_days_user_1[test_load_less_seven_days_user_1['sex']==0]
# aa['sex_0'] = 2
# bb = test_load_less_seven_days_user_1[test_load_less_seven_days_user_1['sex']==1]
# bb['sex_1'] = 1
# cc = test_load_less_seven_days_user_1[test_load_less_seven_days_user_1['sex']==2]
# cc['sex_2'] = 3
# test_load_less_seven_days_user_1 = pd.concat([aa,bb,cc],axis=0)
# test_load_less_seven_days_user_1 = test_load_less_seven_days_user_1.fillna(0)

#计算时间衰减度，先将时间转化为数字
# test_load_less_seven_days_user_1["time0"]=test_load_less_seven_days_user_1.time.apply(lambda x : int(x[5:7])-3)
# test_load_less_seven_days_user_1['time1'] = test_load_less_seven_days_user_1.time.apply(lambda x: int(x[8:10]))
# test_load_less_seven_days_user_1['time2'] = test_load_less_seven_days_user_1.time.apply(lambda x: int(x[11:13]))
# test_load_less_seven_days_user_1['time3'] = test_load_less_seven_days_user_1.time.apply(lambda x: int(x[14:16]))
test_load_less_seven_days_user_1

,8/count,age,age_0,age_1,age_2,age_3,age_4,browse_count,cate_8,load_days,...,sex_1,sex_2,time,type,user_id,user_lv_cd,time0,time1,time2,time3
3299,0.177419,-1,2,0,0,0,0,62.0,11.0,6.0,...,0.0,0.0,2016-04-07 09:55:13,1,210931,4,1,7,9,55
3300,0.177419,-1,2,0,0,0,0,62.0,11.0,6.0,...,0.0,0.0,2016-04-07 09:55:13,6,210931,4,1,7,9,55
3301,0.177419,-1,2,0,0,0,0,62.0,11.0,6.0,...,0.0,0.0,2016-04-07 09:55:13,6,210931,4,1,7,9,55
3302,0.177419,-1,2,0,0,0,0,62.0,11.0,6.0,...,0.0,0.0,2016-04-07 09:55:13,1,210931,4,1,7,9,55
3303,0.177419,-1,2,0,0,0,0,62.0,11.0,6.0,...,0.0,0.0,2016-04-07 09:55:24,5,210931,4,1,7,9,55
3304,0.177419,-1,2,0,0,0,0,62.0,11.0,6.0,...,0.0,0.0,2016-04-07 09:55:24,5,210931,4,1,7,9,55
3305,0.177419,-1,2,0,0,0,0,62.0,11.0,6.0,...,0.0,0.0,2016-04-08 14:56:43,6,210931,4,1,8,14,56
7705,1.000000,-1,2,0,0,0,0,17.0,17.0,3.0,...,0.0,0.0,2016-04-04 11:21:49,1,228905,5,1,4,11,21
7706,1.000000,-1,2,0,0,0,0,17.0,17.0,3.0,...,0.0,0.0,2016-04-04 11:21:49,6,228905,5,1,4,11,21
7707,1.000000,-1,2,0,0,0,0,17.0,17.0,3.0,...,0.0,0.0,2016-04-04 11:24:58,6,228905,5,1,4,11,24


In [12]:
#用户数据清洗
def search_user_step_1(user_dataset):
    #有过第8类商品购买记录的清除掉
    delet_user_id = pd.DataFrame(user_dataset[(user_dataset['cate']==8)&(user_dataset['type']==4)]['user_id']).drop_duplicates()
    delet_user_id['label'] = [x for x in range(len(delet_user_id))]
    retain_user_id = pd.merge(user_dataset,delet_user_id,on='user_id',how='left')
    retain_user_id = retain_user_id.fillna(0)
    retain_user_id = pd.DataFrame(retain_user_id[retain_user_id['label']==0]['user_id']).drop_duplicates()
    return retain_user_id

def search_user_step_2(user_id):
    #清除所有用户等级低于3和age在46-55岁的用户
    user_info = pd.read_csv(USER_FILE,encoding='gbk')
    user_data = pd.merge(user_id,user_info,on='user_id',how='left')
    user_data = user_data[user_data['user_lv_cd']>=3]
    retain_user_id = pd.DataFrame(user_data[user_data['age']!='46-55岁']['user_id'])
    return retain_user_id


def search_user_step_3(buy_user_id):
    if os.path.exists('./cache/Action_2016-02-01_2016-04-11-buy_user_load_days'):
        with open('./cache/Action_2016-02-01_2016-04-11-buy_user_load_days','rb') as f:
            user_load_info = pickle.load(f)
    else:
        with open('./cache/Action_2016-02-01_2016-04-11-load_days','rb') as f:                   #训练集用用,最后测试改一下
            user_load_info = pickle.load(f)
        user_load_info = pd.merge(buy_user_id,user_load_info,on='user_id',how='left')
        with open('./cache/Action_2016-02-01_2016-04-11-buy_user_load_days','wb') as f:
            pickle.dump(user_load_info,f)
    return user_load_info

def search_user_step_4():
    #选取登陆天数小于七天，并且在最后7天浏览了商品的用户
    dump_path = './cache/Action_2016-02-01_2016-04-11-load_less_eight_days'
    if os.path.exists(dump_path):
        with open(dump_path,'rb') as f:
            user_dataset = pickle.load(f)
    else:
        user_load_info = pd.read_csv('load_frame.csv')
        with open('./cache/Action_2016-02-01_2016-04-11-user_id-cate-time-type','rb') as f:
            user_dataset = pickle.load(f)
        user_load_info = user_load_info[['user_id','load_days','last_buy_time']]
        user_load_info = user_load_info[user_load_info['load_days']<=7]
        user_dataset = user_dataset[user_dataset['cate']==8]
        user_dataset = pd.merge(user_load_info,user_dataset,on='user_id',how='left')
        user_dataset = user_dataset[user_dataset['time']>'2016-04-09 00:00:00']
        with open(dump_path,'wb') as f:
            pickle.dump(user_dataset,f)
    return user_dataset

In [13]:
b =search_user_step_4()
b

,user_id,load_days,sku_id,time,cate,type
6,245667.0,2.0,49103,2016-04-05 16:40:21,8,6
7,245667.0,2.0,49103,2016-04-05 16:40:21,8,6
8,245667.0,2.0,49103,2016-04-05 16:40:21,8,1
9,245667.0,2.0,49103,2016-04-05 16:40:21,8,6
10,245667.0,2.0,49103,2016-04-05 16:40:21,8,1
459,284981.0,7.0,164954,2016-04-04 21:39:33,4,1
460,284981.0,7.0,164954,2016-04-04 21:39:33,4,6
461,284981.0,7.0,164954,2016-04-04 21:39:34,4,6
615,287367.0,7.0,103451,2016-04-04 20:48:57,9,1
616,287367.0,7.0,103451,2016-04-04 20:48:57,9,6


In [81]:
# with open('./cache/Action_2016-02-01_2016-04-11-user_id-cate-time-type','rb') as f:
#     user_dataset = pickle.load(f)
# a = search_user_step_1(user_dataset=user_dataset)
# b = search_user_step_2(user_id=a)
# search_user_step_3(buy_user_id=b)

,user_id,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,...,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,load_days
0,263587.0,1,1,0,0,0,1,0,0,0,...,0,1,0,0,1,0,1,0,0,20.0
1,217892.0,1,1,1,1,1,1,1,1,1,...,1,1,1,0,0,0,0,1,1,49.0
2,283139.0,1,1,1,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,13.0
3,243907.0,1,1,0,0,0,0,0,0,0,...,1,1,0,0,1,1,0,0,0,17.0
4,209278.0,1,1,1,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,1,19.0
5,217567.0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,10.0
6,207684.0,1,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,21.0
7,272629.0,0,1,1,1,0,0,0,0,1,...,1,1,1,1,1,1,0,1,1,41.0
8,284046.0,0,1,0,0,0,0,0,0,0,...,1,1,1,0,1,0,0,1,1,23.0
9,208720.0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,8.0


In [68]:
# delet_user_id = pd.DataFrame(user_dataset[(user_dataset['cate']==8)&(user_dataset['type']==4)]['user_id']).drop_duplicates()
# retain_user_id = pd.merge(user_dataset,delet_user_id,on='user_id',how='left')

# retain_user_id = retain_user_id.fillna()
# retain_user_id = pd.DataFrame(retain_user_id[retain_user_id['type']==0]['user_id']).drop_duplicates()

,user_id
3211,280086.0
19602,304798.0
22790,247280.0
46820,229437.0
64171,298237.0
68593,235628.0
72647,224102.0
73905,261538.0
76561,263319.0
81102,222651.0
